In [1]:
from typing import Any, Callable, Dict, Optional
from numpy import ndarray
from scipy.sparse import csc_matrix
from typing import Union
from typing import Optional
from warnings import warn
import osqp
from numpy import hstack, inf, ndarray, ones
from osqp import OSQP
from scipy import sparse
import numpy as np


In [2]:
DenseOrCSCMatrix = Union[ndarray, csc_matrix]

def osqp_solve_qp( P, q, G, h, A, b,initvals: Optional[DenseOrCSCMatrix] = None, 
                  verbose: bool = False, eps_abs: float = 1e-4, eps_rel: float = 1e-4, 
                  polish: bool = True, **kwargs) -> Optional[ndarray]:
   
    if isinstance(P, ndarray):
        P = sparse.csc_matrix(P)
    solver = OSQP()
    kwargs = { "eps_abs": eps_abs, "eps_rel": eps_rel, "polish": polish, "verbose": verbose,}
    
    if isinstance(A, ndarray):
            A = sparse.csc_matrix(A)
    l_inf = -inf * ones(len(h))
    qp_A = sparse.vstack([G, A], format="csc")
    qp_l = hstack([l_inf, b])
    qp_u = hstack([h, b])
    solver.setup(P=P, q=q, A=qp_A, l=qp_l, u=qp_u, **kwargs)
    
    
    if initvals is not None:
        solver.warm_start(x=initvals)
    res = solver.solve()
    if hasattr(solver, "constant"):
        success_status = solver.constant("OSQP_SOLVED")
    else:  # more recent versions of OSQP
        success_status = osqp.constant("OSQP_SOLVED")
    if res.info.status_val != success_status:
        warn(f"OSQP exited with status '{res.info.status}'")
        return None
    return res.x

In [3]:
m = 5
n = 700
p = 5
P = np.loadtxt('P.txt')
q = np.loadtxt('q.txt')
G = np.loadtxt('G.txt')
h = np.loadtxt('h.txt')
A = np.loadtxt('A.txt')
b = np.loadtxt('b.txt')


x=osqp_solve_qp( P,q, G,h,A,b)
v=0.5*x.T@P@x+q.T@x
print(v)

-42.31408315828458
